## Advise re identity architecture

Initialize

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.0.1"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.0.1-preview"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.0.1-alpha"

#!import ../config/Settings.cs
#!import ../Plugins/EmailPlugin.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Planning.Handlebars;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Plugins.Core;

var (useAzureOpenAI, model, azureEndpoint, apiKey, bingApiKey, orgId) = Settings.LoadFromFile();
var kernel = Microsoft.SemanticKernel.Kernel.CreateBuilder()
            .AddAzureOpenAIChatCompletion(
                model,   // deployment name
                azureEndpoint, // Azure OpenAI Endpoint
                apiKey)      // Azure OpenAI Key
            .Build();

Installed Packages Microsoft.SemanticKernel, 1.0.1 Microsoft.SemanticKernel.Planners.Handlebars, 1.0.1-preview Microsoft.SemanticKernel.Plugins.Core, 1.0.1-alpha

Load plugins

In [2]:
kernel.Plugins.Clear();
var pluginPath = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "..", "Plugins");
foreach (var plugin in new string[] { "IdentityPlugin" })
{
    Console.WriteLine($"Loading ${pluginPath}${plugin}");
    kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginPath, plugin));
}

Loading $c:\Users\mrochon\source\repos\ai-samples\NotebookSK\8. Identity\..\Plugins$IdentityPlugin


Create and save plan

In [3]:
var goal = File.ReadAllText("SimpleSystem.txt");
#pragma warning disable SKEXP0060
var planner = new HandlebarsPlanner();
var plan = await planner.CreatePlanAsync(kernel, goal);
Console.WriteLine("Plan:\n");
Console.WriteLine(plan.ToString());
File.WriteAllText("./plan.txt", plan.ToString());

Plan:

{{!-- Step 1: Create variable to store the system description --}}
{{set "systemDescription" "a single web application showing html content"}}

{{!-- Step 2: Call IdentityPlugin-Applications helper to get the list of application types --}}
{{set "applicationTypes" (IdentityPlugin-Applications)}}

{{!-- Step 3: Print the system description --}}
System Description: {{systemDescription}}

{{!-- Step 4: Loop through the application types and print them --}}
Application Types:
{{#each applicationTypes}}
- {{this}}
{{/each}}


If needed, modify plan in ./plan.txt, then load and execute plan with a new goal.

In [7]:
#pragma warning disable SKEXP0060
var myPlan = new HandlebarsPlan(File.ReadAllText("./plan.txt"));
#pragma warning disable SKEXP0060
var resp = await plan.InvokeAsync(kernel);
Console.WriteLine("Result:\n");
Console.WriteLine(resp);

Result:

System Description: a single web application showing html content

Application Types:
- 44
